In [1]:
import sys
import os
import numpy as np
import random as rand
import math
from pathlib import Path
from multiprocessing import Pool
from functools import partial

In [2]:
def readInput(filename):
    with open(filename) as f:
        numWizards = int(f.readline())
#         trueOrder = f.readline().split(" ")
        numConstraints = int(f.readline())
        constraints = []
        wizards = set()
        for _ in range(numConstraints):
            c = f.readline().split()
            constraints.append(c)
            for w in c:
                wizards.add(w)

    wizards = list(wizards)
    wizards.sort()
    dictWizards = {}
    for i in range(len(wizards)):
        dictWizards[wizards[i]] = i
    
    for i in constraints:
        for j in range(3):
            i[j] = dictWizards[i[j]]
        if int(i[0]) > int(i[1]):
            i[0], i[1] = i[1], i[0]
    return numWizards, numConstraints, list(range(numWizards)), constraints

def displayInput(filename):
    n, c, names, constraints = readInput(filename)
    string = str(n) + "\n"
    string += " ".join(names) + "\n"
    string += str(c) + "\n"
    for i in constraints:
        string += " ".join(i) + "\n"
    print(string)

In [3]:
# assigment is a dict: wix->index, constraints: list((a,b,c) ... (x,y,z))
def h(assigment, constraints):
    conflicts = 0
    for constraint in constraints:
        try:
            t0, t1 = assigment[constraint[0]], assigment[constraint[1]]
        except:
            print(constraint)
            print(len(assigment), assigment)
        a = t0 if (t0 < t1) else t1
        b = t0 + t1 - a
        c = assigment[constraint[2]]
        if a<c and c<b:
            conflicts += 1
    return conflicts

In [4]:
#currently gives a random assigment
def getAssigment(numWiz, wizs):
    indecies = list(range(1,numWiz+1))
    rand.shuffle(indecies)
    ass = {wizs[i]:indecies[i] for i in range(len(wizs))}
    return ass

In [5]:
#Hill climbing solver
def solve(wizs, constraints):
    numWiz = len(wizs)
    numCons = len(constraints)
    ass = getAssigment(numWiz, wizs)
    conflicts = h(ass, constraints) 
    while conflicts > 0:
        best_score = conflicts
        for i in range(numWiz):
            for j in range(i+1,numWiz):
                x = ass[wizs[i]]
                y = ass[wizs[j]]
                ass[wizs[i]] = y
                ass[wizs[j]] = x

                newScore = h(ass, constraints)
                if newScore == 0:
                    return ass
                if newScore < best_score:
                    best_ass = ass.copy()
                    best_score = newScore
                    
                ass[wizs[i]] = x
                ass[wizs[j]] = y
        if best_score >= conflicts:
            ass = getAssigment(numWiz, wizs)
            conflicts = h(ass, constraints) 
        else:
            conflicts = best_score
            ass = best_ass.copy()

In [6]:
# Run hill climbing, takes a while to complete for large input!!!!!!!!!!!

# %time sol = solve(wizs, constraints)
# print('conflicts', h(sol, constraints))
# for key in sol:
#     print (key, sol[key])

In [7]:
# Calculate probabily for given state
def acceptance_prob(oldCost, newCost, T):
    exponent = (oldCost - newCost)/(1.0*T)
#   if exponent > 0 probability will be >1. just return 1 to prevent crazy big numbers.
    if exponent > 0:
        return 1
    p = math.e**exponent
    return p

In [8]:
def anneal(wizs, constraints, start, bestAss=None):
    numWiz = len(wizs)
    numCons = len(constraints)
    ass = start.copy()
    conflicts = h(ass, constraints) 
    
    bestScore = 500
    if bestAss == None:
        bestAss = [start.copy()]
    
#     meta parameters, T should be 1, 0.8<alpha<0.99, 100<iterations<1000 should work well
    T = 1.0
    T_min = 0.00001
    alpha = 0.9
    iterations = 500
    while conflicts > 0 and T > T_min:
        for i in range(iterations):
#         Choosing 2 random wizards
            wizA, wizB = rand.sample(wizs, 2)
            
            x = ass[wizA]
            y = ass[wizB]
            ass[wizA] = y
            ass[wizB] = x
            newScore = h(ass, constraints)
#             if solution found, return
            if newScore == 0:
                return ass, None
#             keeping track of best solutions found
            if newScore == bestScore:
                bestAss.append(ass.copy())
            elif newScore < bestScore:
                bestScore = newScore
                bestAss = [ass.copy()]
#                 calculating probability
            ap = acceptance_prob(conflicts, newScore, T)
#             save change
            if ap > rand.random(): 
                conflicts = newScore
#             revert change
            else: 
                ass[wizA] = x
                ass[wizB] = y
#         reduce tempreture
        T *= alpha
    return bestAss[0], bestAss

In [9]:
%%time 
numWiz, numCons, wizs, constraints = readInput('inputs50/input50_0.in')
start = wizs.copy()
rand.shuffle(start)
print('initial number of conflicts:', h(start, constraints))
sol, bestAss = anneal(wizs, constraints, start)
print('solution conflicts', h(sol, constraints))
# for key in sol:
#     print (key, sol[key])
print('best score:', h(sol, constraints))

initial number of conflicts: 163
solution conflicts 57
best score: 57
Wall time: 27 s


In [15]:
reinterpret(200,0, [78, 6, 61, 164, 186, 49, 129, 13, 46, 50, 121, 168, 102, 64, 4, 100, 111, 80, 197, 166, 153, 55, 30, 110, 54, 136, 98, 185, 89, 40, 124, 188, 192, 154, 27, 68, 180, 73, 175, 108, 161, 93, 182, 123, 17, 28, 172, 167, 145, 9, 88, 176, 69, 53, 75, 198, 29, 181, 187, 14, 24, 152, 15, 183, 2, 115, 37, 36, 58, 137, 44, 134, 94, 133, 92, 10, 97, 42, 103, 0, 106, 170, 81, 66, 128, 74, 151, 150, 41, 142, 130, 105, 189, 159, 21, 114, 38, 57, 125, 8, 96, 190, 193, 77, 144, 173, 122, 163, 67, 135, 16, 34, 56, 104, 22, 23, 138, 1, 11, 47, 132, 19, 157, 178, 141, 79, 158, 51, 131, 70, 59, 72, 118, 60, 12, 148, 191, 169, 194, 117, 120, 43, 82, 18, 184, 52, 5, 31, 112, 86, 160, 165, 143, 177, 3, 45, 162, 139, 76, 32, 146, 155, 101, 62, 126, 107, 71, 83, 91, 84, 171, 116, 109, 35, 90, 174, 26, 20, 127, 119, 63, 113, 39, 199, 179, 147, 48, 87, 149, 196, 156, 140, 7, 65, 33, 95, 85, 99, 195, 25])

Jaden Logan Gayle Robby Beth Otis Alan Ursula Kedrick Dolores Henry Lois Michael Archie Fay Gabriella Lavana Delores Nikolas Louise Suzette Katherine Leroy Lesa Frances Willie Susan Cynthia Dianne Emerald Cam Pandora Rudy Vickie Leah Stephen Gilbert George Katrina Thelma Clifford Jodi Howard Neal Hanna Rod Armando Loretta Tony Anna Arrie Marsha Noel Dulce Carter Byron Leilani Kayla Giselle Mekhi Micah Alannah Shawn Tayla Beau Van Jeanette Laura Daniella Douglas Matthew Shelia Melinda Darius Jenny Elaina Rosie Kris Ahmad Malia Bradley Jean Niamh Silas Skylar Wanda Raquel Trent Donald Ciara Sterling Skye Harry Dashee Harper Victor Kendrick Homer Cheyenne Wilfred Bonita Sarah Bart Isiah Leo Karla Jaiden Shelby Darragh Stacy Camila Brad Quinn Taylor Katie Gena Sophie Nasir Melissa Tatiana Natalie Aurora Kyler Delbert Colleen Kaylin Shayla Tadhg Jeanine Annabella Joseph Matt Lorraine Harrison Hannah Laurel Casey Graham Lester Roosevelt Umar Lynnette Jonas Rhonda Kristi Dmitri Sammy Tina Mig

In [14]:
def reinterpret(num, idx, seq):
    with open('Staff_Inputs/staff_' + str(num) + '.in') as f:
        numWizards = int(f.readline())
#         trueOrder = f.readline().split(" ")
        numConstraints = int(f.readline())
        constraints = []
        wizards = set()
        for _ in range(numConstraints):
            c = f.readline().split()
            constraints.append(c)
            for w in c:
                wizards.add(w)

    wizards = list(wizards)
    wizards.sort()
    dictWizards = {}
    for i in range(len(wizards)):
        dictWizards[i] = wizards[i]
    newOrder = [0] * len(seq)
    for i in range(len(seq)):
        newOrder[seq[i]] = dictWizards[i]
    string = ""
    for i in newOrder:
        string += i + " "
    print(string)
    f = open('outputs/staff_' + str(num) + '.out', 'w')
    f.write(string)
    f.close()

In [13]:
input_size = 50
numWiz = [0] * 10
numCons = [0] * 10
wizs = [0] * 10
Constraints = [0] * 10
bestAss = []
pool = []

for i in range(10):
    string = 'inputs' + str(input_size) + '/input' + str(input_size) + '_' + str(i)
    numWiz[i], numCons[i], wizs[i], constraints[i] = readInput(string + '.in')
    bestAss.append([wizs[i].copy()])
    rand.shuffle(bestAss[i][0])
    f = Path(string + ".out")
    if not f.is_file():
        pool.append(i)
print("Problems remaining:", pool)

Problems remaining: [0, 1, 2, 9]


In [12]:
while len(pool) > 0:
    for i in pool:
        start = wizs[i].copy()
        rand.shuffle(start)
        sol, retAss = anneal(wizs[i], constraints[i], start)
        if retAss == None:
            pool.remove(i)
            print(i, sol)
            reinterpret(input_size, i, sol)
            break
        else:
            bestAss[i] = retAss

[15, 41, 48]
50 [10, 47, 1, 38, 14, 34, 49, 32, 23, 44, 41, 26, 0, 35, 30, 19, 4, 3, 21, 24, 25, 7, 37, 27, 18, 20, 31, 16, 43, 36, 29, 22, 9, 2, 17, 42, 40, 45, 39, 11, 33, 15, 8, 48, 46, 5, 28, 6, 12, 13]
[15, 16, 19]
50 [11, 47, 2, 37, 8, 33, 48, 32, 23, 44, 41, 27, 0, 35, 30, 21, 1, 16, 22, 24, 25, 7, 49, 29, 18, 19, 31, 4, 43, 36, 28, 20, 9, 3, 17, 42, 40, 45, 38, 12, 34, 15, 10, 39, 46, 5, 26, 6, 13, 14]
5 [25, 22, 34, 21, 28, 42, 49, 46, 6, 20, 39, 38, 0, 33, 41, 27, 3, 45, 1, 18, 24, 19, 7, 2, 44, 37, 11, 17, 36, 35, 48, 10, 43, 12, 9, 47, 29, 30, 26, 4, 31, 15, 14, 13, 32, 5, 40, 16, 23, 8]
Elyza Deborah Kale Bailey Shannon Benita Alicia Tammi Wayne Matilda Kendall Kurt Timmy Gerardo Ebony Suzanne Trinity Cecelia Neal Dakota Bradwin Cathy Taylor Neil Michael Darius Grady Aden Paulina Kathy Ronan Chloe Cathleen Gloria Garrett Dolores Wanda Callum Vladimir Salvatore Henry Nelly Alexa Isabella Jasmine Orlando Aimee Mohamed Elijah Kolby 


UnsupportedOperation: not writable

In [ ]:
%%time
partial_aneal = partial(anneal, wizs, constraints)
p = Pool()
starts = [getAssigment(numWiz, wizs) for _ in range(100)]
results = p.map(partial_aneal, starts)